In [263]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.table import Table, Column
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [264]:
def toList(df,columns):
    idx =df.index.tolist()
    l=[]
    for i in idx:
        temp=[]
        for j in columns:
            a=df.get_value(i,j)
            temp.append(a)
        l.append(temp)
    return l

In [265]:
class NavieBaye:
    def __init__(self):
        pass


    def __calculateIndividualProbability__(self,y):
        value,count=self.__getUnique__(y)
        return np.divide(count,len(y))
    
    def __getUnique__(self,lis):
        y=lis
        indexlist=y.index.tolist()
        temp=[]
        for i in indexlist:
            temp.append(y.get(i))
        uniqueValueSet=set(temp)

        countList=[]
        for j in uniqueValueSet:
            c=[]
            counter=0
            for i in indexlist:
                if y.get(i)==j:
                    counter+=1
            countList.append(counter)

        return (list(uniqueValueSet),countList)
    
    def __constructProbabilities__(self,X,y):
        tiger=[] # values of each attributes 
        tables=[] # tables is the probabilty table
        features=X.columns
    #     converting to series if y is a dataframe
        if isinstance(y, pd.DataFrame):
            y=y[y.columns[0]]

        uniqueValueInY,valueCountListInY=self.__getUnique__(y)

        
        for index,i in enumerate(features):
            t=Table()
            uniqueValue,count=self.__getUnique__(X[i])
            yak=[]
            for ind,j in enumerate(uniqueValue):
                total=[]
                for jack,k in enumerate(uniqueValueInY):# this uniqueValueSet is of y
                    if ind==0:
                        if jack==0:
                            co=Column(name=i,dtype='S20')
                            t.add_column(co)
                        col=Column(name=k)
                        t.add_column(col)
                        
                    abc=len(X.ix[((X[i]==j) & (y == k))])
                    total.append(abc)
                ashod=np.divide(total,valueCountListInY)
                ashod=ashod.tolist()
                ashod.insert(0, j)
                t.add_row(ashod)
                yak.append(total)
            tiger.append(yak)
            tables.append(t)
        return tiger,tables
        
    def __assignLabel__(self,y):
        
        pass
    
    
    def fit(self,X,y):
#         X and y is the dataframe
        self.columns=X.columns
        
        self.values,self.probability=self.__constructProbabilities__(X,y)
        self.individualProbability=self.__calculateIndividualProbability__(y)
        
    def __predictProbability__(self,totalSample):    
        finalList=[]
        for saIndex,sample in enumerate(totalSample):
            total=[]
            for sampleIndex,eachAttribute in enumerate(sample):
                probTable=self.probability[sampleIndex]
                tempList={}
                for rowIndex,row in enumerate(probTable):
                    if row[0].decode('UTF-8')==str(eachAttribute):
                        lis=[]
                        for colIndex in range(0,len(probTable.colnames)):
                            if colIndex!=0:
                                lis.append(row[colIndex])
                        tempList[eachAttribute]=lis
                        break
                total.append(tempList)
            finalList.append(total)
        return finalList
    
    def __findMaxWithIndex__(self,a):
        max_=a[0]
        maxIndex = 0
        for i in range(len(a)):
            if a[i] > max_:
                max_ = a[i]
                maxIndex = i
        return maxIndex,max_
    
    def predictClass(self,totalSample):
        finalList=self.__predictProbability__(totalSample)
        classes=[]
        ans=[]
        
        
        def createClassesList():
            temp=[]
            for i in finalList[0]:
                va=list(i.values())[0]
                for j in va:
                    temp.append([])
                break
            return temp
        
        classes=createClassesList()
        
        
        for listIndex,element in enumerate(finalList):
            temp=[]
            for j in element:
                listOfValues=list(j.values())[0]
                for classesIndex,cla in enumerate(classes):
                    cla.append(listOfValues[classesIndex])
            
            decision=[]
            for classesIndex,cla in enumerate(classes):
                yak=np.prod(cla)*self.individualProbability[classesIndex]
                decision.append(yak)
                
                
            maxIndex,max_=self.__findMaxWithIndex__(decision)
            
            lis=self.probability[0].colnames
            
            ans.append(lis[maxIndex+1])
            classes=createClassesList()
        return ans

In [266]:
df=pd.read_csv('Gallon.csv')
df.sample()

,mpg,cylinders,displacement,horsepower,weight,acceleration,modelyear,maker
8,Bad,8,260,110,4060,19.0,77,America


### Here you do not divide the data into train test split. Since there are chance that the output class of any attribute may not be present in train but present in test due to split

In [267]:
features=df.columns[:-1]
target=df.columns[-1]
X=df[features]
y=df[target]


In [268]:
nb=NavieBaye()
nb.fit(X,y)

In [269]:
nb=NaiveBayes()

In [270]:
nb.fit(X,y)

### after fitting the data you can see the probability table for each attribute i.e. each column

In [271]:
nb.probability[0]

mpg,Europe,Asia,America
bytes20,float64,float64,float64
Bad,0.0,0.272727272727,0.695652173913
Good,0.375,0.181818181818,0.130434782609
Ok,0.625,0.545454545455,0.173913043478


In [272]:
nb.probability[1]

cylinders,Europe,Asia,America
bytes20,float64,float64,float64
8,0.0,0.0,0.304347826087
4,0.875,0.909090909091,0.347826086957
6,0.125,0.0909090909091,0.347826086957


In [273]:
nb.probability[2].show_in_notebook()

idx,displacement,Europe,Asia,America
0,258,0.0,0.0,0.0434782608696
1,260,0.0,0.0,0.0434782608696
2,134,0.125,0.0,0.0
3,135,0.0,0.0,0.0869565217391
4,140,0.0,0.0,0.0434782608696
5,400,0.0,0.0,0.0869565217391
6,146,0.125,0.0,0.0
7,151,0.0,0.0,0.0434782608696
8,156,0.0,0.0,0.0434782608696
9,163,0.0,0.0909090909091,0.0


In [274]:
nb.probability[3].show_in_notebook()

idx,horsepower,Europe,Asia,America
0,130,0.0,0.0,0.0434782608696
1,133,0.0,0.0909090909091,0.0
2,145,0.0,0.0,0.0434782608696
3,150,0.0,0.0,0.0869565217391
4,170,0.0,0.0,0.0434782608696
5,175,0.0,0.0,0.0434782608696
6,60,0.125,0.0909090909091,0.0
7,61,0.125,0.0,0.0
8,63,0.0,0.0,0.0869565217391
9,68,0.125,0.0,0.0


In [275]:
nb.probability[4].show_in_notebook()

idx,weight,Europe,Asia,America
0,3840,0.0,0.0,0.0434782608696
1,2945,0.0,0.0909090909091,0.0
2,2560,0.125,0.0,0.0
3,1925,0.0,0.0909090909091,0.0
4,4746,0.0,0.0,0.0434782608696
5,3085,0.0,0.0,0.0434782608696
6,1940,0.0,0.0909090909091,0.0
7,2074,0.0,0.0909090909091,0.0
8,3613,0.0,0.0,0.0434782608696
9,4385,0.0,0.0,0.0434782608696


In [276]:
nb.probability[5].show_in_notebook()

idx,acceleration,Europe,Asia,America
0,12.0,0.0,0.0,0.0869565217391
1,13.8,0.125,0.0,0.0
2,14.4,0.0,0.0,0.0434782608696
3,14.2,0.125,0.0,0.0
4,16.9,0.0,0.0909090909091,0.0
5,12.5,0.0,0.0,0.0434782608696
6,14.5,0.125,0.272727272727,0.0434782608696
7,14.0,0.0,0.0909090909091,0.0434782608696
8,15.8,0.0,0.0909090909091,0.0
9,15.4,0.0,0.0,0.0434782608696


In [277]:
nb.probability[6].show_in_notebook()

idx,modelyear,Europe,Asia,America
0,71,0.0,0.181818181818,0.0434782608696
1,72,0.125,0.0909090909091,0.0869565217391
2,73,0.0,0.181818181818,0.0
3,74,0.125,0.0909090909091,0.0869565217391
4,75,0.25,0.0909090909091,0.173913043478
5,76,0.0,0.0909090909091,0.0869565217391
6,77,0.0,0.0909090909091,0.0869565217391
7,78,0.125,0.0909090909091,0.0434782608696
8,79,0.0,0.0909090909091,0.0869565217391
9,80,0.0,0.0,0.0434782608696


In [278]:
l=toList(test[features],test[features].columns)

In [279]:
testData=[['Bad',8,304,150,3892,12.5,72],['Good',4,89,71,1925,12.5,81]]
lis=nb.predictClass(testData)

In [280]:
lis

['America', 'Europe']